In [15]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
nlp = spacy.load('nl_core_news_sm')
import warnings
warnings.filterwarnings('ignore')

In [17]:
tfidf_df = pd.read_csv("tfidf_df.csv")

In [18]:
tfidf_df.sort_values(by='similarity', ascending=False, inplace=True)
tfidf_df.reset_index(inplace=True, drop=True)

In [19]:
tfidf_m = pd.DataFrame(np.sort(tfidf_df.iloc[:, :2])).duplicated()

In [20]:
tfidf_df = tfidf_df[~tfidf_m]
tfidf_df.reset_index(inplace=True, drop=True)

In [83]:
# tfidf_df.head()

In [23]:
tfidf_df.reset_index(inplace=True, drop=True)
tfidf_df.shape

(65118, 3)

In [81]:
tfidf_df['similarity'].describe()

count    59710.000000
mean         0.908759
std          0.103992
min          0.600081
25%          0.860320
50%          0.945232
75%          1.000000
max          1.000000
Name: similarity, dtype: float64

In [62]:
for index in range(len(tfidf_df)):
    index, r_ = index, tfidf_df.iloc[[index]].values.flatten().tolist()
    df_ = tfidf_df.loc[(tfidf_df.daltix_id_1 == r_[1]) | (tfidf_df.daltix_id_2 == r_[1])]
    if len(df_) > 3:
        break

In [63]:
index, r_

(196,
 ['191999e11f63f7ff7af62a78520c23962533d0bb91e16a0a53fe16b5ecb5e87e',
  'fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce234f4e20073d639f16',
  1.0000000000000009])

In [73]:
# temp_ = tfidf_df[tfidf_df.apply(lambda r: r.str.match(r_[0], case=False).any(), axis=1)] 
# temp_ = temp_[temp_.index != index] 
# temp_

In [65]:
temp_ = tfidf_df[tfidf_df.apply(lambda r: r.str.match(r_[1], case=False).any(), axis=1)] 
temp_ = temp_[temp_.index != index] 
temp_

,daltix_id_1,daltix_id_2,similarity
241,1695e7b780a833e56a4a0fb1ed9683ae1e674fd295a3a3...,fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce...,1.000000
251,1c175ea287a8a53b444ecb6902b98fd40efcf8a051cfe5...,fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce...,1.000000
15069,465ff30ea53e62080b85ffd8e2a2ee312ee4ed34184274...,fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce...,0.998752
15232,66bea80a97be93291313c3db78e892e1f54c2a4d347eee...,fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce...,0.997883
15253,63800632d2c4153eb40f4ff31bf87d8b3706cdd2484001...,fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce...,0.997795


In [77]:

doc0 = df.loc[df['DALTIX_ID'] == r_[0]]['text'].item()
doc1 = df.loc[df['DALTIX_ID'] == r_[1]]['text'].item()
doc0_, doc1_ = nlp(doc0), nlp(doc1)

print(r_[0], r_[2], doc1_.similarity(doc0_))
print(doc0)
print()

print(r_[1], r_[2], doc0_.similarity(doc1_))
print(doc1)
print()

for index, row in temp_.iterrows():
    if row['daltix_id_1'] == r_[1]:
        doc2 = df.loc[df['DALTIX_ID'] == row['daltix_id_2']]['text'].item()
        sim_0, sim_2  = doc1_.similarity(doc0_), doc1_.similarity(nlp(doc2))
        print(row['daltix_id_2'], row['similarity'], sim_0, sim_2)
    else:
        doc2 = df.loc[df['DALTIX_ID'] == row['daltix_id_1']]['text'].item()
        sim_0, sim_2  = doc1_.similarity(doc0_), doc1_.similarity(nlp(doc2))
        print(row['daltix_id_1'], row['similarity'], sim_0, sim_2)
    print(df.loc[df['DALTIX_ID'] == row['daltix_id_1']]['text'].item())
    print()


191999e11f63f7ff7af62a78520c23962533d0bb91e16a0a53fe16b5ecb5e87e 1.0000000000000009 1.0
buis biedt vele toepassingsmogelijkheden bouwen opmaken renoveren versieren element creatief design concepten gebruikt worden gebruikt opbouw poortjes hekken bovendien zowel binnen buitenshuis gebruikt  gah alberts ronde buis aluminium grijs 0,6 cm aantal stuks afwerking geanodiseerd breedte mm dikte mm gebruik meerdere gebruiken kleur grijs lengte 100 cm materiaal aluminium type buis vorm ronde buis zelfklevend span class mxd icon mxd icon small mxd icon reverse mxd icon only mxd icon cancel mxd icon colored data reactid 22nee span

fd6f1ba947ed971f72c7dbbf90a09a367b710d086257ce234f4e20073d639f16 1.0000000000000009 1.0
buis biedt vele toepassingsmogelijkheden bouwen opmaken renoveren versieren element creatief design concepten gebruikt worden gebruikt opbouw poortjes hekken bovendien zowel binnen buitenshuis gebruikt  gah alberts ronde buis aluminium grijs 0,6 cm aantal stuks afwerking geanodiseerd

From above EDA I have got to know that there aren't only paird similar products. There are grouped similar products. <br>
eg: (a, b),(c, b),(b, e) <br>
Therefor if a == b & c == b & c == b then a == c & c == b <br>
So result should be (a, b),(c, b),(b, e)(c, b)(a, c) <br>

In [ ]:
# this to change submition dataset as explaind above

new_entries = []
threshold = 0.99
for index in tqdm(range(len(tfidf_df))):
    index, r_ = index, tfidf_df.iloc[[index]].values.flatten().tolist()
    
    df_0 = tfidf_df.loc[(tfidf_df.daltix_id_1 == r_[0]) | (tfidf_df.daltix_id_2 == r_[0])]
    df_1 = df_0[df_0.similarity > threshold]
    s_2 = set(np.concatenate((df_0.daltix_id_1.unique(), df_0.daltix_id_2.unique())).tolist())
    new_entries = new_entries + list(itertools.combinations(s_2, 2))
    
    df_1 = tfidf_df.loc[(tfidf_df.daltix_id_1 == r_[1]) | (tfidf_df.daltix_id_2 == r_[1])]
    df_1 = df_1[df_1.similarity > threshold]
    s_1 = set(np.concatenate((df_1.daltix_id_1.unique(), df_1.daltix_id_2.unique())).tolist())
    new_entries = new_entries + list(itertools.combinations(s_1, 2))
#     if len(df_1) > 2:
#         break
    
len(new_entries)
new_entries = pd.DataFrame(new_entries, columns=['daltix_id_1', 'daltix_id_2'])
new_entries.to_csv("sub_new_entries.csv", index=False)
sub_new_entries = pd.read_csv("sub_new_entries.csv")

In [ ]:
df = pd.concat([submission_df,sub_new_entries]).drop_duplicates()
df.reset_index(inplace=True, drop=True)
df_ = pd.DataFrame(np.sort(df.iloc[::])).duplicated()
submission_df = df[~df_]
len(submission_df)

Above changes to datasets didnt gave better results..

In [ ]:
# this to change validation dataset as same as submition dataset

df_val = pd.read_csv("data/y_true.csv")
df_val.columns = ['daltix_id_1', 'daltix_id_2']
df_val.shape
new_entries = []
for index in tqdm(range(len(df_val))):
    index, r_ = index, df_val.iloc[[index]].values.flatten().tolist()
    
    df_0 = df_val.loc[(df_val.daltix_id_1 == r_[0]) | (df_val.daltix_id_2 == r_[0])]
    s_2 = set(np.concatenate((df_0.daltix_id_1.unique(), df_0.daltix_id_2.unique())).tolist())
    new_entries = new_entries + list(itertools.combinations(s_2, 2))
    
    df_1 = df_val.loc[(df_val.daltix_id_1 == r_[1]) | (df_val.daltix_id_2 == r_[1])]
    s_1 = set(np.concatenate((df_1.daltix_id_1.unique(), df_1.daltix_id_2.unique())).tolist())
    new_entries = new_entries + list(itertools.combinations(s_1, 2))
len(new_entries)
new_entries = pd.DataFrame(new_entries, columns=['daltix_id_1', 'daltix_id_2'])
new_entries.to_csv("val_new_entries.csv", index=False)
val_new_entries = pd.read_csv("val_new_entries.csv")

In [ ]:
df = pd.concat([df_val,val_new_entries]).drop_duplicates()
df.reset_index(inplace=True, drop=True)
df_ = pd.DataFrame(np.sort(df.iloc[::])).duplicated()
df_val = df[~df_]
len(df_val)

### Dataframe testings

In [ ]:
raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jacobson', 'Miller'], 
        'last_name': ['Miller', 'Jacobson', 'Ali', 'Molly', 'Jason'], 
        'age': [1.0, 0.52, 0.36, 1.0, 0.72]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age'])

In [ ]:
df.sort_values(by='age', ascending=False, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df

In [ ]:
# df_ = df[df.apply(lambda r: r.str.match('Jason', case=False).any(), axis=1)]
# df_[df_.index != 2]

In [ ]:
m = pd.DataFrame(np.sort(df.iloc[:, :2])).duplicated()
df = df[~m]

In [ ]:
df

In [ ]:
raw_data_ = {'first_name': [ 'Ali'], 
        'last_name': ['Spa'], 
        'age': [0.89]}
df_ = pd.DataFrame(raw_data_, columns = ['first_name', 'last_name', 'age'])

In [ ]:
df_

In [ ]:
raw_data = {'first_name': ['Jas', 'Mol', 'Tina', 'Jas', 'Miller', 'Mill'], 
        'last_name': ['Miller', 'Jacob', 'Ali', 'Molly', 'Jas', 'Jas'], 
        'age': [1.0, 0.52, 0.36, 1.0, 0.72, 0.23]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age'])

In [ ]:
df

In [ ]:
df_ = pd.DataFrame(np.sort(df.iloc[:, :2])).duplicated()
df_

In [ ]:
df[~df_]

In [ ]:
df_ = df.loc[(df.last_name == 'Jas') | (df.first_name == 'Jas')]
df_

In [ ]:
# df_ = df.loc[(df.last_name == 'Jas') | (df.first_name == 'Jas')].loc[df.age < 0.9]
# df_

In [ ]:
pd.merge(df,df_, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [ ]:
df1 = pd.DataFrame(data = {'col1' : ['Jason', 'Molly', 'Tina', 'Mill'], 'col2' : ['Miller', 'Jacobson', 'Ali', 'Jas']}) 
df2 = pd.DataFrame(data = {'col1' : [ 'Molly', 'Tina', 'Miller'], 'col2' : ['Jacobson', 'Apa', 'Jason']})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df = pd.concat([df1,df2]).drop_duplicates()
df.reset_index(inplace=True, drop=True)
df_ = pd.DataFrame(np.sort(df.iloc[::])).duplicated()
df[~df_]

In [ ]:
s=pd.concat([df1,df2])
s[s.duplicated()]

In [ ]:
df2 = df2[['col2','col1']]
df2.columns = ['col1', 'col2']
s=pd.concat([df1,df2])
s[s.duplicated()]